# 시작

In [25]:
import os, glob, re
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 데이터 준비

In [26]:
def read_lyrics():
    '''
    파일에서 가사들을 불러옴, LMS동일
    '''
    dir_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
    result = []
    for file_path in glob.glob(dir_path):
        with open(file_path, 'r') as lyric_file:
            sentences = lyric_file.read().splitlines()
            result.extend(sentences)
    return result

def preprocess_sentence(sentence):
    '''
    문장들에서 특수 문자, 띄어쓰기 등을 보정한 후 <start>와 <end>를 문장 처음과 끝에 추가, LMS동일
    '''
    sentence = sentence.lower().strip()  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

def make_corpus(sentences):
    corpus = []
    for sentence in sentences:
        if len(sentence) == 0: continue
        if sentence[-1] == ":": continue
        corpus.append(preprocess_sentence(sentence))
    return corpus

In [27]:
def tokenize(corpus, num_words=12000):
    '''
    문장을 단어 단위로 끊고, 토큰으로 생성함, LMS와 거의 같음
    
    corpus    : 문장
    num_words : 사용할 단어 수
    
    return    : 생성된 토큰 텐서, 토큰화에 사용된 맵
    '''
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = list(filter(lambda x: len(x) < 16, tensor)) # 토큰의 길이를 15개로 제한
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    return tensor, tokenizer

In [28]:
def make_single_dataset(x, y, batch_size):
    '''
    한 개의 데이터셋을 생성. tf.data.Dataset
    '''
    size = len(x)
    return tf.data.Dataset.from_tensor_slices((x, y)).shuffle(size).batch(batch_size, drop_remainder=True)
    
def make_dataset(source, target, batch_size=256):
    '''
    훈련, 검증 데이터셋을 생성. tf.data.Dataset
    
    source : 문장의 시작
    target : 문장의 다음
    
    return : 훈련 데이터셋, 검증 데이터셋
    '''
    train_x, test_x, train_y, test_y = train_test_split(source, target, test_size=0.2, shuffle=True)
    print(train_x.shape)
    print(train_y.shape)
    train_dataset = make_single_dataset(train_x, train_y, batch_size)
    val_dataset = make_single_dataset(test_x, test_y, batch_size)
    
    return train_dataset, val_dataset

# 모델

In [29]:
'''
학습에 사용할 모델
'''
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, dropout=0.2, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, dropout=0.2, return_sequences=True)
        # self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        # self.dense_1 = tf.keras.layers.Dense(vocab_size * 2, activation='softmax')
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        # out = self.rnn_3(out)
        out = self.linear(out)
        return out

In [30]:
def make_model(sample, vocab_size=12001, embedding_size=256, hidden_size=1024):
    '''
    모델을 생성함
    
    sample         : 모델의 입력 텐서 차원을 알기 위한 샘플
    vocab_size     : 단어의 가짓 수
    embedding_size : 단어 임베딩의 깊이
    hidden_size    : 모델 내부의 hidden layer의 수
    
    return         : 모델
    '''
    model = TextGenerator(vocab_size, embedding_size , hidden_size)
    model(sample)
    model.summary()
    
    optimizer = tf.keras.optimizers.Nadam()
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    model.compile(loss=loss, optimizer=optimizer)
    return model

# 테스트

In [31]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    '''
    문장이 어떻게 생성되는지 시험, LMS동일
    
    model         : 학습이 완료된 모델
    tokenizer     : 문장을 토큰화 하는데 사용한 맵
    init_sentence : 문장의 시작
    max_len       : 최대 문장의 단어 수
    
    return        : 생성된 문장
    '''
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

# 메인

데이터 준비

In [32]:
num_words = 20000
raw_sentences = read_lyrics()
corpus = make_corpus(raw_sentences)
tensor, tokenizer = tokenize(corpus, num_words)

src_input = tensor[:,:-1]
tgt_input = tensor[:,1:]

train_dataset, val_dataset = make_dataset(src_input, tgt_input, batch_size=128)

(124810, 14)
(124810, 14)


모델 생성

In [33]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=1024, hidden_size=1024)

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  20481024  
_________________________________________________________________
lstm_6 (LSTM)                multiple                  8392704   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  20501025  
Total params: 57,767,457
Trainable params: 57,767,457
Non-trainable params: 0
_________________________________________________________________


모델 훈련

In [34]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
975/975 [==============================] - 177s 181ms/step - loss: 3.2588 - val_loss: 2.9237
Epoch 2/10
975/975 [==============================] - 176s 180ms/step - loss: 2.7833 - val_loss: 2.7087
Epoch 3/10
975/975 [==============================] - 176s 181ms/step - loss: 2.5212 - val_loss: 2.5732
Epoch 4/10
975/975 [==============================] - 176s 181ms/step - loss: 2.2961 - val_loss: 2.4786
Epoch 5/10
975/975 [==============================] - 176s 181ms/step - loss: 2.1037 - val_loss: 2.4101
Epoch 6/10
975/975 [==============================] - 177s 181ms/step - loss: 1.9389 - val_loss: 2.3636
Epoch 7/10
975/975 [==============================] - 178s 182ms/step - loss: 1.7984 - val_loss: 2.3336
Epoch 8/10
975/975 [==============================] - 177s 182ms/step - loss: 1.6783 - val_loss: 2.3160
Epoch 9/10
975/975 [==============================] - 180s 184ms/step - loss: 1.5751 - val_loss: 2.3105
Epoch 10/10
975/975 [==============================] - 180s 184m

생성해보기

In [35]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

----------------------

모델 생성

In [36]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=512, hidden_size=1024)

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  10240512  
_________________________________________________________________
lstm_8 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_9 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_4 (Dense)              multiple                  20501025  
Total params: 45,429,793
Trainable params: 45,429,793
Non-trainable params: 0
_________________________________________________________________


모델 훈련

In [ ]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
975/975 [==============================] - 136s 140ms/step - loss: 3.2806 - val_loss: 2.9299
Epoch 2/10
975/975 [==============================] - 133s 137ms/step - loss: 2.8036 - val_loss: 2.7129
Epoch 3/10
975/975 [==============================] - 133s 137ms/step - loss: 2.5225 - val_loss: 2.5730
Epoch 4/10
975/975 [==============================] - 136s 139ms/step - loss: 2.2869 - val_loss: 2.4731
Epoch 5/10
975/975 [==============================] - 134s 137ms/step - loss: 2.0887 - val_loss: 2.4075
Epoch 6/10
975/975 [==============================] - 137s 140ms/step - loss: 1.9233 - val_loss: 2.3647
Epoch 7/10
975/975 [==============================] - 134s 138ms/step - loss: 1.7846 - val_loss: 2.3362
Epoch 8/10
264/975 [=======>......................] - ETA: 1:31 - loss: 1.6348

생성해보기

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

----------------------

모델 생성

In [ ]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=512, hidden_size=512)

모델 훈련

In [ ]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

생성해보기

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

----------------------

모델 생성

In [ ]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=256, hidden_size=512)

모델 훈련

In [ ]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

생성해보기

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

----------------------

모델 생성

In [ ]:
for src_sample, tgt_sample in train_dataset.take(1): break
model = make_model(src_sample, vocab_size=num_words+1, embedding_size=256, hidden_size=256)

모델 훈련

In [ ]:

model.fit(train_dataset, validation_data=val_dataset, epochs=10)

생성해보기

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

----------------------